# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
import os
import numpy as np

import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

from glob import glob
import sys


In [3]:
# Write your code below.
%load_ext dotenv
%dotenv ../../01_materials/labs/.env

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")

pq_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(pq_files).set_index("ticker")


In [5]:
dd_px.compute().head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613010,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281464,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782814,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344181,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372852,2819626,Health Care,Life Sciences Tools & Services,2000
A,2000-01-10,49.356224,49.803291,48.327969,49.311516,41.757935,2148446,Health Care,Life Sciences Tools & Services,2000
A,2000-01-11,49.311516,49.311516,47.523247,48.640915,41.190056,1855985,Health Care,Life Sciences Tools & Services,2000
A,2000-01-12,48.640915,48.640915,45.824390,47.657368,40.357174,1429874,Health Care,Life Sciences Tools & Services,2000
A,2000-01-13,48.909157,49.937412,47.210300,48.372677,40.962921,1134337,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Write your code below.

dd_feat = ( dd_px.groupby('ticker', group_keys=False).apply( lambda x: x.assign( Close_lag_1 = x['Close'].shift(1) ) ) )
dd_feat = ( dd_feat.groupby('ticker', group_keys=False).apply( lambda x: x.assign( hi_lo_range = x['High'] - x['Low'] ) ) )
dd_feat = ( dd_feat.groupby( 'ticker', group_keys=False ).apply( lambda x: x.assign( Adj_Close_lag_1 = x['Adj Close'].shift(1) ) ) .assign(returns = lambda x: x['Adj Close']/x['Adj_Close_lag_1'] - 1 ) )


/var/folders/d7/l8wqbmzx3m74cqmpn7nsvgch0000gn/T/ipykernel_4151/862505081.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat =   (  dd_px.groupby('ticker', group_keys=False).apply( lambda x: x.assign( Close_lag_1 = x['Close'].shift(1) ) ) )
/var/folders/d7/l8wqbmzx3m74cqmpn7nsvgch0000gn/T/ipykernel_4151/862505081.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (  dd_feat.groupby('ticker', group_keys=False).apply( lambda x: x.assign( hi_lo_range = x['High'] - x['Low'] ) ) )
/var/folders/d7/l8wqb

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,hi_lo_range,Adj_Close_lag_1,returns
ticker,,,,,,,,,,,,,,
AME,2005-01-03,10.607407,10.657778,10.545185,10.562963,9.324914,818775,Industrials,Electrical Components & Equipment,2005,NaN,0.112593,NaN,NaN
AME,2005-01-04,10.554074,10.598519,10.047407,10.290370,9.084269,904500,Industrials,Electrical Components & Equipment,2005,10.562963,0.551112,9.324914,-0.025807
AME,2005-01-05,10.290370,10.290370,10.071111,10.097778,8.914246,1370250,Industrials,Electrical Components & Equipment,2005,10.290370,0.219259,9.084269,-0.018716
AME,2005-01-06,10.100741,10.228148,10.047407,10.183704,8.990103,1195425,Industrials,Electrical Components & Equipment,2005,10.097778,0.180741,8.914246,0.008510
AME,2005-01-07,10.213333,10.222222,10.077037,10.177778,8.984873,662513,Industrials,Electrical Components & Equipment,2005,10.183704,0.145185,8.990103,-0.000582
AME,2005-01-10,10.074074,10.343704,10.074074,10.240000,9.039802,878175,Industrials,Electrical Components & Equipment,2005,10.177778,0.269630,8.984873,0.006113
AME,2005-01-11,10.201481,10.281481,10.059259,10.192593,8.997950,1480950,Industrials,Electrical Components & Equipment,2005,10.240000,0.222221,9.039802,-0.004630
AME,2005-01-12,10.195556,10.308148,10.035556,10.293333,9.086884,1462725,Industrials,Electrical Components & Equipment,2005,10.192593,0.272593,8.997950,0.009884
AME,2005-01-13,10.293333,10.370370,10.204444,10.254815,9.052877,915638,Industrials,Electrical Components & Equipment,2005,10.293333,0.165926,9.086884,-0.003742


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [7]:
# Write your code below.
pd_df = dd_feat.compute()
pd_df['Rolling_return_Avg'] = pd_df.groupby('ticker')['returns'].transform(lambda x: x.rolling(window=10 , min_periods=10, center=True).mean())
pd_df.head(25)


#It wasn't necessary to convert the dask dataframe to pandas. 
#Because the dataframe doesn't need to be rendered until the final compute, applying the transformation would likely have been faster with Dask

DataError: Cannot aggregate non-numeric type: datetime64[ns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.